In [1]:
pip install pywin32 openpyxl pyodbc


In [2]:
import win32com.client
import openpyxl
import pyodbc

# 1. HWP 파일 읽기
def read_hwp(file_path):
    hwp = win32com.client.Dispatch("HWPFrame.HwpObject")
    hwp.RegisterModule("FilePathCheckDLL", "SecurityModule")
    hwp.Open(file_path, "", "HWP")  # ★ 여기 3개 인수

    text = hwp.GetTextFile("Text", "")
    hwp.Quit()
    return text

def read_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    data = []
    for line in lines:
        parts = line.strip().split(maxsplit=1)
        if len(parts) == 2:
            data.append((parts[0], parts[1]))
    return data



# 2. 텍스트 가공
def parse_text(text):
    lines = text.strip().split('\n')
    data = []
    for line in lines:
        parts = line.strip().split(maxsplit=1)  # 첫 단어와 나머지 구분
        if len(parts) == 2:
            term, definition = parts
            data.append((term.strip(), definition.strip()))
    return data

# 3. Excel 저장
def save_to_excel(data, excel_path):
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "Dictionary"

    ws.append(["Term", "Definition"])
    for term, definition in data:
        ws.append([term, definition])

    wb.save(excel_path)
    print(f"Saved to Excel: {excel_path}")

# 4. Access 저장
def save_to_access(data, access_path):
    conn_str = (
        r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
        f'DBQ={access_path};'
    )
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # 테이블 생성
    try:
        cursor.execute("""
        CREATE TABLE Dictionary (
            Term TEXT,
            Definition TEXT
        )
        """)
    except Exception as e:
        print(f"Table may already exist: {e}")

    # 데이터 삽입
    for term, definition in data:
        cursor.execute(
            "INSERT INTO Dictionary (Term, Definition) VALUES (?, ?)", 
            (term, definition)
        )
    conn.commit()
    cursor.close()
    conn.close()
    print(f"Saved to Access: {access_path}")

# 실행
hwp_file = r"C:\Users\YOON\1_202502_python_project\Food_dic_1.hwp"
excel_file = r"C:\Users\YOON\1_202502_python_project\Food_dic_1.xlsx"
access_file = r"C:\Users\YOON\1_202502_python_project\Food_dic_1.accdb"

text = read_hwp(hwp_file)
data = parse_text(text)
save_to_excel(data, excel_file)
save_to_access(data, access_file)


AttributeError: 'NoneType' object has no attribute 'strip'

In [5]:
import openpyxl
import pyodbc

# 1. 텍스트 파일 읽기 함수
def read_txt(file_path):
    with open(file_path, 'r', encoding='euc-kr') as f:  # ★ euc-kr 로 변경
        lines = f.readlines()
    data = []
    for line in lines:
        parts = line.strip().split(maxsplit=1)
        if len(parts) == 2:
            data.append((parts[0], parts[1]))
    return data


# 2. Excel 저장 함수
def save_to_excel(data, excel_path):
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "Dictionary"

    ws.append(["Term", "Definition"])
    for term, definition in data:
        ws.append([term, definition])

    wb.save(excel_path)
    print(f"✅ Excel 파일 저장 완료: {excel_path}")

# 3. Access DB 저장 함수
def save_to_access(data, access_path):
    conn_str = (
        r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
        f'DBQ={access_path};'
    )
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # 테이블 생성 (있으면 생략하거나 try-except)
    try:
        cursor.execute("""
        CREATE TABLE Dictionary (
            Term TEXT,
            Definition TEXT
        )
        """)
    except Exception as e:
        print(f"테이블이 이미 존재할 수 있습니다: {e}")

    # 데이터 삽입
    for term, definition in data:
        cursor.execute(
            "INSERT INTO Dictionary (Term, Definition) VALUES (?, ?)", 
            (term, definition)
        )
    conn.commit()
    cursor.close()
    conn.close()
    print(f"✅ Access DB 저장 완료: {access_path}")

# 4. 실행부
if __name__ == "__main__":
    txt_file = r"C:\Users\YOON\1_202502_python_project\20241001_EUC_KR_Additive_4.txt"
    excel_file = r"C:\Users\YOON\1_202502_python_project\20241001_EUC_KR_Additive_4.xlsx"
    #access_file = r"C:\Users\YOON\1_202502_python_project\Food_dic_1.accdb"

    # 텍스트 파일 읽기
    data = read_txt(txt_file)

    # Excel 저장
    save_to_excel(data, excel_file)

    # Access DB 저장
    #save_to_access(data, access_file)

    print("🎉 변환 작업이 모두 완료되었습니다!")


✅ Excel 파일 저장 완료: C:\Users\YOON\1_202502_python_project\20241001_EUC_KR_Additive_4.xlsx
🎉 변환 작업이 모두 완료되었습니다!


In [6]:
import openpyxl
import pyodbc

# 1. 텍스트 파일 읽기 (Term, English, Definition 분리)
def read_txt(file_path):
    with open(file_path, 'r', encoding='euc-kr') as f:
        lines = f.readlines()
    data = []
    for line in lines:
        parts = line.strip().split(maxsplit=2)  # 앞 2개 단어 + 나머지
        if len(parts) == 3:
            term, english, definition = parts
            data.append((term.strip(), english.strip(), definition.strip()))
    return data

# 2. Excel 저장 함수 (3열 구성)
def save_to_excel(data, excel_path):
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "Dictionary"

    ws.append(["Term", "English", "Definition"])  # 헤더
    for term, english, definition in data:
        ws.append([term, english, definition])

    wb.save(excel_path)
    print(f"✅ Excel 파일 저장 완료: {excel_path}")

# 3. Access 저장 함수 (원하면 여기도 3열로 수정 가능)
def save_to_access(data, access_path):
    conn_str = (
        r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
        f'DBQ={access_path};'
    )
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # 테이블 생성
    try:
        cursor.execute("""
        CREATE TABLE Dictionary (
            Term TEXT,
            English TEXT,
            Definition TEXT
        )
        """)
    except Exception as e:
        print(f"테이블이 이미 존재할 수 있습니다: {e}")

    # 데이터 삽입
    for term, english, definition in data:
        cursor.execute(
            "INSERT INTO Dictionary (Term, English, Definition) VALUES (?, ?, ?)",
            (term, english, definition)
        )
    conn.commit()
    cursor.close()
    conn.close()
    print(f"✅ Access DB 저장 완료: {access_path}")

# 4. 실행부
if __name__ == "__main__":
    txt_file = r"C:\Users\YOON\1_202502_python_project\20241001_EUC_KR_Fungi_4.txt"
    excel_file = r"C:\Users\YOON\1_202502_python_project\20241001_EUC_KR_Additive_4_B.xlsx"
    #access_file = r"C:\Users\YOON\1_202502_python_project\Food_dic_1.accdb"

    # 텍스트 파일 읽기
    data = read_txt(txt_file)

    # Excel 저장
    save_to_excel(data, excel_file)

    # Access 저장
    #save_to_access(data, access_file)

    print("🎉 변환 작업이 모두 완료되었습니다!")


✅ Excel 파일 저장 완료: C:\Users\YOON\1_202502_python_project\20241001_EUC_KR_Additive_4_B.xlsx
🎉 변환 작업이 모두 완료되었습니다!


In [17]:
import openpyxl
import pyodbc
import re

# 한글 포함 여부 확인 함수
def contains_korean(s):
    return any('\uac00' <= c <= '\ud7a3' for c in s)

# 텍스트 파일 읽기 함수
def read_txt(file_path):
    with open(file_path, 'r', encoding='euc-kr') as f:
        lines = f.readlines()
    
    data = []
    for line in lines:
        tokens = line.strip().split()
        if not tokens:
            continue

        term = tokens[0]  # 첫 단어는 한글 용어
        english_tokens = []
        definition_tokens = []

        for token in tokens[1:]:
            if contains_korean(token):
                # 첫 한글 등장 이후부터는 정의
                definition_tokens = tokens[tokens.index(token):]
                break
            english_tokens.append(token)
        
        english = ' '.join(english_tokens)
        definition = ' '.join(definition_tokens)
        data.append((term, english, definition))

    return data

# 엑셀 저장 함수
def save_to_excel(data, excel_path):
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "Dictionary"
    ws.append(["Term", "English", "Definition"])
    for term, english, definition in data:
        ws.append([term, english, definition])
    wb.save(excel_path)
    print(f"✅ Excel 파일 저장 완료: {excel_path}")

# Access DB 저장 함수
def save_to_access(data, access_path):
    conn_str = (
        r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
        f'DBQ={access_path};'
    )
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    try:
        cursor.execute("""
        CREATE TABLE Dictionary (
            Term TEXT,
            English TEXT,
            Definition TEXT
        )
        """)
    except Exception as e:
        print(f"테이블이 이미 존재할 수 있습니다: {e}")

    for term, english, definition in data:
        cursor.execute(
            "INSERT INTO Dictionary (Term, English, Definition) VALUES (?, ?, ?)",
            (term, english, definition)
        )
    conn.commit()
    cursor.close()
    conn.close()
    print(f"✅ Access DB 저장 완료: {access_path}")

# 실행부
if __name__ == "__main__":
    txt_file = r"C:\Users\YOON\1_202502_python_project\20241001_EUC_KR_Fungi_3.txt"
    excel_file = r"C:\Users\YOON\1_202502_python_project\20241001_EUC_KR_Fungi_3_C.xlsx"
    #access_file = r"C:\Users\YOON\1_202502_python_project\Food_dic_1.accdb"

    data = read_txt(txt_file)
    save_to_excel(data, excel_file)
    #save_to_access(data, access_file)

    print("🎉 모든 작업 완료!")


✅ Excel 파일 저장 완료: C:\Users\YOON\1_202502_python_project\20241001_EUC_KR_Fungi_3_C.xlsx
🎉 모든 작업 완료!
